In [2]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
import xgboost as xgb
import numpy as np
import os
os.chdir("/Users/alexanderschou/Desktop/Projects/adv_mla_2024/AT2_experimentation/data/processed")

In [3]:
sales_train_processed = pd.read_csv('sales_train_processed.csv')

/var/folders/mh/j3zy8xqx2v32dk0tv4nx5x8m0000gn/T/ipykernel_3565/3300575340.py:1: DtypeWarning: Columns (9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  sales_train_processed = pd.read_csv('sales_train_processed.csv')


In [4]:
X = sales_train_processed.drop(columns=['revenue'])
y = sales_train_processed['revenue']
categorical_columns = ['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'event_name', 'event_type']
numerical_columns = ['sales', 'year', 'week', 'sell_price']

In [6]:
categorical_preprocessor = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='None')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

numerical_preprocessor = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value=0))
])

In [8]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_preprocessor, numerical_columns),
        ('cat', categorical_preprocessor, categorical_columns)
    ])

In [11]:
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', xgb.XGBRegressor(objective='reg:squarederror', random_state=42))
])

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
pipeline.fit(X_train, y_train)

In [ ]:
val_score = pipeline.score(X_val, y_val)

In [ ]:
print(f"Validation score: {val_score}")

In [13]:
X[["item_id","store_id","date"]]

,item_id,store_id,date
0,HOBBIES_1_001,CA_1,2011-01-29
1,HOBBIES_1_002,CA_1,2011-01-29
2,HOBBIES_1_003,CA_1,2011-01-29
3,HOBBIES_1_004,CA_1,2011-01-29
4,HOBBIES_1_005,CA_1,2011-01-29
...,...,...,...
47107045,FOODS_3_823,WI_3,2015-04-18
47107046,FOODS_3_824,WI_3,2015-04-18
47107047,FOODS_3_825,WI_3,2015-04-18
47107048,FOODS_3_826,WI_3,2015-04-18


In [ ]:
import joblib

pipeline.fit(X[["item_id","store_id","date"]], y)
os.chdir("/Users/alexanderschou/Desktop/Projects/adv_mla_2024/AT2_experimentation/models/predictive")
joblib.dump(pipeline, 'xgboost_model_new.joblib')

In [ ]:
X.tail(5)

In [ ]:
imputation_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor)
])

imputation_pipeline.fit(X)

os.chdir("/Users/alexanderschou/Desktop/Projects/adv_mla_2024/AT2_experimentation/models/predictive")
joblib.dump(imputation_pipeline, 'imputation_model.joblib')

In [10]:
import joblib
joblib.dump(imputation_pipeline, 'imputation_model.joblib')

['imputation_model.joblib']